In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Data transformations (with augmentation for training data)
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224 (input size of ResNet)
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization (ImageNet stats)
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),  # Resize images to 224x224
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Set up data directories and loaders
data_dir = r'path to\clip\kill_or_no_kill'  # Replace with your dataset directory
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification (kill banner vs no kill banner)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: kill and no_kill

# Transfer the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Make sure the model is on the selected device

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)  # Move the criterion to the device
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop function
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)  # Move inputs to the selected device (GPU or CPU)
                labels = labels.to(device)  # Move labels to the selected device (GPU or CPU)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        print()

    return model

# Train the model
model = train_model(model, criterion, optimizer, num_epochs=10)

# Save the trained model
torch.save(model.state_dict(), 'kill_banner_model.pth')


Epoch 1/10
----------
train Loss: 0.1580 Acc: 0.9383
val Loss: 0.1825 Acc: 0.8961

Epoch 2/10
----------
train Loss: 0.0236 Acc: 0.9941
val Loss: 0.0920 Acc: 0.9740

Epoch 3/10
----------
train Loss: 0.0073 Acc: 0.9980
val Loss: 0.0009 Acc: 1.0000

Epoch 4/10
----------
train Loss: 0.0010 Acc: 1.0000
val Loss: 0.0007 Acc: 1.0000

Epoch 5/10
----------
train Loss: 0.0008 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 6/10
----------
train Loss: 0.0003 Acc: 1.0000
val Loss: 0.0001 Acc: 1.0000

Epoch 7/10
----------
train Loss: 0.0004 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 8/10
----------
train Loss: 0.0002 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 9/10
----------
train Loss: 0.0004 Acc: 1.0000
val Loss: 0.0000 Acc: 1.0000

Epoch 10/10
----------
train Loss: 0.0215 Acc: 0.9971
val Loss: 0.0184 Acc: 1.0000

